# Example creating custom models with GPT-2




In [15]:
# Import various things
import neural_nlp
from neural_nlp.models import model_pool, model_layers, implementations
from brainscore.utils import LazyLoad
import numpy as np
from transformers import GPT2Tokenizer, GPT2Model

In [24]:
def gpt2_test():
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2Model.from_pretrained(
        "gpt2", output_hidden_states=True
    )
    wrapper = implementations._PytorchTransformerWrapper(
        identifier="gpt2-test",
        tokenizer = tokenizer,
        model=model,
        tokenizer_special_tokens=('ġ',),
        layers=list(("drop",) + tuple(f"encoder.h.{i}" for i in range(12))),
        sentence_average=implementations.word_last
    )
    return wrapper

In [25]:
model_pool["gpt2-test"] = LazyLoad(lambda: gpt2_test())
model_layers["gpt2-test"] = list(("drop",) + tuple(f"encoder.h.{i}" for i in range(12)))
assert model_layers["gpt2-test"] == model_layers["gpt2"]


In [26]:
print(GPT2Tokenizer.from_pretrained("gpt2"))

In [27]:
# Run and evaluate the model generated in gpt_test()
# NOTE THIS CELL TAKES A REALLY LONG TIME
from neural_nlp import score

score(benchmark="Pereira2018-encoding", model="gpt2-test")

layers:   0%|          | 0/13 [00:00<?, ?it/s]Using cls_token, but it is not set yet.

token features: 100%|██████████| 66/66 [00:00<00:00, 176.68it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 280.49it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 72/72 [00:00<00:00, 212.62it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 277.40it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 56/56 [00:00<00:00, 208.02it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 266.27it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 62/62 [00:00<00:00, 179.25it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 273.93it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 60/60 [00:00<00:00, 178.04it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 278.76it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 49/49 [00:00<00

token features: 100%|██████████| 59/59 [00:00<00:00, 181.07it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 276.08it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 46/46 [00:00<00:00, 168.01it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 272.49it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 42/42 [00:00<00:00, 174.15it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 274.68it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 56/56 [00:00<00:00, 188.73it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 271.61it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 47/47 [00:00<00:00, 175.53it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 281.23it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 53/53 [00:00<00:00, 173.03it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 283.08it/s]
Usi

token features: 100%|██████████| 50/50 [00:00<00:00, 185.16it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 287.96it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 56/56 [00:00<00:00, 177.49it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 270.62it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 56/56 [00:00<00:00, 183.41it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 285.88it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 57/57 [00:00<00:00, 168.19it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 283.47it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 52/52 [00:00<00:00, 182.96it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 281.37it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 37/37 [00:00<00:00, 173.35it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 281.87it/s]
Usi


token features: 100%|██████████| 56/56 [00:00<00:00, 189.37it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 277.82it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 51/51 [00:00<00:00, 187.49it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 279.46it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 64/64 [00:00<00:00, 175.09it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 272.92it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 77/77 [00:00<00:00, 182.75it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 275.54it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 61/61 [00:00<00:00, 174.53it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 273.92it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 52/52 [00:00<00:00, 163.18it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 267.63it/s]
Us

token features: 100%|██████████| 57/57 [00:00<00:00, 181.49it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 277.29it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 68/68 [00:00<00:00, 177.90it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 273.85it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 65/65 [00:00<00:00, 171.57it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 278.82it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 59/59 [00:00<00:00, 179.28it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 290.37it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 53/53 [00:00<00:00, 186.23it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 284.70it/s]
Using cls_token, but it is not set yet.

token features: 100%|██████████| 67/67 [00:00<00:00, 177.63it/s]

layer packaging: 100%|██████████| 13/13 [00:00<00:00, 284.16it/s]
Usi

...

cross-validation:  80%|████████  | 4/5 [01:17<00:19, 19.41s/it]

cross-validation: 100%|██████████| 5/5 [01:37<00:00, 19.41s/it]

{'experiment': '243sentences', 'atlas': 'visual'}:  80%|████████  | 8/10 [09:21<02:17, 68.58s/it]
{'experiment': '243sentences', 'atlas': 'auditory'}:  80%|████████  | 8/10 [09:21<02:17, 68.58s/it]

cross-validation:   0%|          | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|██        | 1/5 [00:02<00:11,  2.81s/it]

cross-validation:  40%|████      | 2/5 [00:05<00:08,  2.76s/it]

cross-validation:  60%|██████    | 3/5 [00:08<00:05,  2.75s/it]

cross-validation:  80%|████████  | 4/5 [00:11<00:02,  2.77s/it]

cross-validation: 100%|██████████| 5/5 [00:13<00:00,  2.76s/it]

{'experiment': '243sentences', 'atlas': 'auditory'}:  90%|█████████ | 9/10 [09:35<00:51, 51.52s/it]
{'experiment': '243sentences', 'atlas': 'MD'}:  90%|█████████ | 9/10 [09:35<00:51, 51.52s/it]      

cross-validation:   0%|          | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|██      

cross-validation:  20%|██        | 1/5 [00:23<01:32, 23.06s/it]

cross-validation:  40%|████      | 2/5 [00:46<01:09, 23.09s/it]

cross-validation:  60%|██████    | 3/5 [01:09<00:46, 23.03s/it]

cross-validation:  80%|████████  | 4/5 [01:32<00:23, 23.05s/it]

cross-validation: 100%|██████████| 5/5 [01:55<00:00, 23.05s/it]

{'experiment': '384sentences', 'atlas': 'MD'}:  50%|█████     | 5/10 [06:37<07:11, 86.39s/it]
{'experiment': '243sentences', 'atlas': 'language'}:  50%|█████     | 5/10 [06:37<07:11, 86.39s/it]

cross-validation:   0%|          | 0/5 [00:00<?, ?it/s]

cross-validation:  20%|██        | 1/5 [00:06<00:24,  6.21s/it]

cross-validation:  40%|████      | 2/5 [00:12<00:18,  6.25s/it]

cross-validation:  60%|██████    | 3/5 [00:18<00:12,  6.22s/it]

cross-validation:  80%|████████  | 4/5 [00:24<00:06,  6.23s/it]

cross-validation: 100%|██████████| 5/5 [00:31<00:00,  6.23s/it]

{'experiment': '243sentences', 'atlas': 'language'}:  60%|██████    | 6/10 [07:09<04:30, 67.69s/it

{'experiment': '384sentences', 'atlas': 'auditory'}:  30%|███       | 3/10 [04:24<12:00, 102.86s/it]

cross-validation:   0%|          | 0/5 [00:00<?, ?it/s]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  20%|██        | 1/5 [00:04<00:16,  4.14s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  40%|████      | 2/5 [00:08<00:12,  4.16s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is 

cross-validation: 100%|██████████| 5/5 [00:39<00:00,  8.00s/it]

{'experiment': '384sentences', 'atlas': 'DMN'}:  20%|██        | 2/10 [01:31<05:57, 44.63s/it]
{'experiment': '384sentences', 'atlas': 'visual'}:  20%|██        | 2/10 [01:31<05:57, 44.63s/it]

cross-validation:   0%|          | 0/5 [00:00<?, ?it/s]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  20%|██        | 1/5 [00:33<02:15, 33.79s/it]/home/rwoo/Documents/columbia/coms6998-project/.venv/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


cross-validation:  40%|████      | 2/5 [01:07<01:41, 33.90s/it]/home/rwoo/Documents/columbia/coms6998-p

<xarray.Score (layer: 13, aggregation: 2)>
array([[0.04500979, 0.10728886],
       [0.60563093, 0.13442728],
       [0.67513002, 0.02300425],
       [0.63005166, 0.07155539],
       [0.5024158 , 0.08306366],
       [0.48248263, 0.12891596],
       [0.50950414, 0.09476689],
       [0.559599  , 0.10482938],
       [0.63031937, 0.18250217],
       [0.72617442, 0.14373017],
       [0.80979421, 0.12343126],
       [0.88677038, 0.05182417],
       [0.9121889 , 0.11140106]])
Coordinates:
  * layer        (layer) <U12 'drop' 'encoder.h.0' ... 'encoder.h.11'
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:          <xarray.Score (layer: 13, aggregation: 2)>\narray([[0.01433...
    ceiling:      <xarray.Score (aggregation: 3)>\narray([0.31856696, 0.01295...
    description:  ceiling-normalized score
    model:        gpt2-test
    benchmark:    Pereira2018-encoding